In [1]:
%pip install --upgrade scipy
%pip install --upgrade numpy
exit()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 33.8 MB 203 kB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.3
    Uninstalling scipy-1.7.3:
      Successfully uninstalled scipy-1.7.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 17.1 MB 76 kB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6


## MMPOSE INSTALL, RUN BUT IGNORE

We recommend to use a conda environment to install mmpose and its dependencies. And compilers `nvcc` and `gcc` are required.

In [1]:
# check NVCC version
!nvcc -V

# check GCC version
!gcc --version

# check python in conda environment
!which python

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

/usr/local/bin/python


In [2]:
# install dependencies: (use cu111 because colab has CUDA 11.1)
%pip install torch==1.10.0+cu111 torchvision==0.11.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html

# install mmcv-full thus we could use CUDA operators
%pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.10.0/index.html

# install mmdet for inference demo
%pip install mmdet


# # clone mmpose repo
# %rm -rf mmpose
!git clone https://github.com/open-mmlab/mmpose.git
%cd mmpose

# # install mmpose dependencies
%pip install -r requirements.txt

# # install mmpose in develop mode
%pip install -e .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu111/torch1.10.0/index.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
fatal: destination path 'mmpose' already exists and is not an empty directory.
/content/mmpose
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Ignoring dataclasses: markers 'python_version == "3.6"' don't match your environment
  Cloning https://github.com/svenkreiss/poseval.git to /tmp/pip-install-gxoadtfn/poseval_8b0c5c82a6ab4da7a67ca5d219f9d882
  Running command git clone -q https://github.com/svenkreiss/poseval.git /tmp/pip-install-gxoadtfn/poseval_8b0c5c82a6ab4da7a67ca5d219f9d882
  Run

In [3]:
# Check Pytorch installation
import torch, torchvision
import torch.nn as nn

print('torch version:', torch.__version__, torch.cuda.is_available())
print('torchvision version:', torchvision.__version__)

# Check MMPose installation
import mmpose

print('mmpose version:', mmpose.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version

print('cuda version:', get_compiling_cuda_version())
print('compiler information:', get_compiler_version())

torch version: 1.10.0+cu111 True
torchvision version: 0.11.0+cu111
mmpose version: 0.29.0
cuda version: 11.1
compiler information: GCC 7.3


/usr/local/lib/python3.8/dist-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [4]:
from mmcv.cnn import constant_init, normal_init

# BackBones

# MMPOSE BACKBONE CLASS STRUCTURE, RUN BUT IGNORE

In [5]:
import logging
from abc import ABCMeta, abstractmethod

import torch.nn as nn


class BaseBackbone(nn.Module, metaclass=ABCMeta):
    """Base backbone.
    This class defines the basic functions of a backbone. Any backbone that
    inherits this class should at least define its own `forward` function.
    """

    def init_weights(self, pretrained=None, patch_padding='pad'):
        """Init backbone weights.
        Args:
            pretrained (str | None): If pretrained is a string, then it
                initializes backbone weights by loading the pretrained
                checkpoint. If pretrained is None, then it follows default
                initializer or customized initializer in subclasses.
        """
        if isinstance(pretrained, str):
            logger = logging.getLogger()
            load_checkpoint(self, pretrained, strict=False, logger=logger, patch_padding=patch_padding)
        elif pretrained is None:
            # use default initializer or customized initializer in subclasses
            pass
        else:
            raise TypeError('pretrained must be a str or None.'
                            f' But received {type(pretrained)}.')

    @abstractmethod
    def forward(self, x):
        """Forward function.
        Args:
            x (Tensor | tuple[Tensor]): x could be a torch.Tensor or a tuple of
                torch.Tensor, containing input data for forward computation.
        """

# Complete Backbone

In [6]:
class convolutionalBlock(nn.Module):
  def __init__(self,in_channels,channel_size=128,num_channels=3, kernel_size=11):
    super().__init__()
    layers = []
    for i in range(num_channels):
      if i == 0:
        input_channel = in_channels
      else:
        input_channel = channel_size
      layers.append(
          nn.Sequential(
          nn.Conv2d(input_channel, channel_size, kernel_size, padding=(kernel_size - 1) // 2 , bias=False),
          nn.BatchNorm2d(channel_size),
          nn.ReLU(inplace=True)
          )
      )

    self.convolutionModule = nn.Sequential(*layers)

  def forward(self, input):
    output = self.convolutionModule(input)
    return output

In [7]:
from mmpose.models.builder import BACKBONES
@BACKBONES.register_module()
class convolutionPoseMachine(BaseBackbone):
  def __init__(self, in_channels=3, out_channels=17, feature_channels=128, middle_channels=32, num_stages=6, pretrained=None):
    super().__init__()
    self.num_stages = num_stages
    self.stem = nn.Sequential(
        nn.Conv2d(in_channels, 128, 9, padding=4, bias=False),
        nn.BatchNorm2d(128),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3,stride=2,padding=1),
        nn.Conv2d(128, 128, 9, padding=4, bias=False),
        nn.BatchNorm2d(128),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3,stride=2,padding=1),
        nn.Conv2d(128, 128, 9, padding=4, bias=False),
        nn.BatchNorm2d(128),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3,stride=2,padding=1),
        nn.Conv2d(128, 32, 5, padding=2, bias=False),
        nn.BatchNorm2d(32),
        nn.ReLU(inplace=True),
        nn.Conv2d(32, 512, 9, padding=4, bias=False),
        nn.BatchNorm2d(512),
        nn.ReLU(inplace=True),
        nn.Conv2d(512, 512, 1, padding=0, bias=False),
        nn.BatchNorm2d(512),
        nn.ReLU(inplace=True),
        nn.Conv2d(512, out_channels, 1, padding=0, bias=True)
    )
    self.middle = nn.Sequential(
        nn.Conv2d(in_channels, 128, 9, padding=4, bias=False),
        nn.BatchNorm2d(128),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3,stride=2,padding=1),
        nn.Conv2d(128, 128, 9, padding=4, bias=False),
        nn.BatchNorm2d(128),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3,stride=2,padding=1),
        nn.Conv2d(128, 128, 9, padding=4, bias=False),
        nn.BatchNorm2d(128),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3,stride=2,padding=1)
    )
    self.convStage = nn.ModuleList([
        convolutionalBlock(middle_channels+out_channels,channel_size=128,kernel_size=11) for i in range(num_stages-1)
    ])
    self.middleConv = nn.ModuleList([
        nn.Sequential(
            nn.Conv2d(128, middle_channels, 5, padding=2, bias=False),
            nn.BatchNorm2d(middle_channels),
            nn.ReLU(inplace=True)
        ) for i in range(num_stages-1)
    ])
    self.outConv = nn.ModuleList([
        nn.Sequential(
            nn.Conv2d(feature_channels, feature_channels, 1, padding=0, bias=False),
            nn.BatchNorm2d(feature_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(feature_channels, out_channels, 1, bias=True)
        ) for i in range(num_stages-1)
    ])

  def forward(self, input):
    stageOne = self.stem(input)
    stageMiddle = self.middle(input)
    output = []
    output.append(stageOne)
    for i in range(self.num_stages - 1):
      stageSingle = self.convStage[i]
      outputSingle = self.outConv[i]
      inputFeatures = torch.cat([output[-1], self.middleConv[i](stageMiddle)],1)
      convolutionFeatures = stageSingle(inputFeatures)
      outputFeatures = outputSingle(convolutionFeatures)
      output.append(outputFeatures)
    return output

  def init_weights(self, pretrained=None):
        """Initialize the weights in backbone.
        Args:
            pretrained (str, optional): Path to pre-trained weights.
                Defaults to None.
        """

        if pretrained is None:
          for m in self.modules():
                if isinstance(m, nn.Conv2d):
                    normal_init(m, std=0.001)
                elif isinstance(m, nn.BatchNorm2d):
                    constant_init(m, 1)


# Custom Dataset & Training

## Train a pose estimation model on a customized dataset

To train a model on a customized dataset with MMPose, there are usually three steps:
1. Support the dataset in MMPose
1. Create a config
1. Perform training and evaluation

### Add a new dataset

There are two methods to support a customized dataset in MMPose. The first one is to convert the data to a supported format (e.g. COCO) and use the corresponding dataset class (e.g. TopdownCOCODataset), as described in the [document](https://mmpose.readthedocs.io/en/latest/tutorials/2_new_dataset.html#reorganize-dataset-to-existing-format). The second one is to add a new dataset class. In this tutorial, we give an example of the second method.

We first download the demo dataset, which contains 100 samples (75 for training and 25 for validation) selected from COCO train2017 dataset. The annotations are stored in a different format from the original COCO format.



## Download coco tiny

In [8]:
# download dataset
%mkdir data
%cd data
!wget https://download.openmmlab.com/mmpose/datasets/coco_tiny.tar
!tar -xf coco_tiny.tar
%cd ..

mkdir: cannot create directory ‘data’: File exists
/content/mmpose/data
--2022-12-11 21:45:17--  https://download.openmmlab.com/mmpose/datasets/coco_tiny.tar
Resolving download.openmmlab.com (download.openmmlab.com)... 47.252.96.15
Connecting to download.openmmlab.com (download.openmmlab.com)|47.252.96.15|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16558080 (16M) [application/x-tar]
Saving to: ‘coco_tiny.tar.2’

coco_tiny.tar.2     100%[===================>]  15.79M  8.48MB/s    in 1.9s    

2022-12-11 21:45:20 (8.48 MB/s) - ‘coco_tiny.tar.2’ saved [16558080/16558080]

/content/mmpose


In [9]:
# check the annotation format
import json
import pprint

anns = json.load(open('data/coco_tiny/train.json'))

print(type(anns), len(anns))
pprint.pprint(anns[0], compact=True)


<class 'list'> 75
{'bbox': [267.03, 104.32, 229.19, 320],
 'image_file': '000000537548.jpg',
 'image_size': [640, 480],
 'keypoints': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 325, 160, 2, 398,
               177, 2, 0, 0, 0, 437, 238, 2, 0, 0, 0, 477, 270, 2, 287, 255, 1,
               339, 267, 2, 0, 0, 0, 423, 314, 2, 0, 0, 0, 355, 367, 2]}


## Top-down coco tiny dataloader



In [11]:
import json
import os.path as osp
from collections import OrderedDict
import tempfile

import numpy as np

from mmpose.core.evaluation.top_down_eval import (keypoint_nme,
                                                  keypoint_pck_accuracy)
from mmpose.datasets.builder import DATASETS
from mmpose.datasets.datasets.base import Kpt2dSviewRgbImgTopDownDataset


@DATASETS.register_module()
class TopDownCOCOTinyDataset5(Kpt2dSviewRgbImgTopDownDataset):

    def __init__(self,
                 ann_file,
                 img_prefix,
                 data_cfg,
                 pipeline,
                 dataset_info=None,
                 test_mode=False):
        super().__init__(
            ann_file,
            img_prefix,
            data_cfg,
            pipeline,
            dataset_info,
            coco_style=False,
            test_mode=test_mode)

        # flip_pairs, upper_body_ids and lower_body_ids will be used
        # in some data augmentations like random flip
        self.ann_info['flip_pairs'] = [[1, 2], [3, 4], [5, 6], [7, 8], [9, 10],
                                       [11, 12], [13, 14], [15, 16]]
        self.ann_info['upper_body_ids'] = (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10)
        self.ann_info['lower_body_ids'] = (11, 12, 13, 14, 15, 16)

        self.ann_info['joint_weights'] = None
        self.ann_info['use_different_joint_weights'] = False

        self.dataset_name = 'coco_tiny'
        self.db = self._get_db()

    def _get_db(self):
        with open(self.ann_file) as f:
            anns = json.load(f)

        db = []
        for idx, ann in enumerate(anns):
            # get image path
            image_file = osp.join(self.img_prefix, ann['image_file'])
            # get bbox
            bbox = ann['bbox']
            # get keypoints
            keypoints = np.array(
                ann['keypoints'], dtype=np.float32).reshape(-1, 3)
            num_joints = keypoints.shape[0]
            joints_3d = np.zeros((num_joints, 3), dtype=np.float32)
            joints_3d[:, :2] = keypoints[:, :2]
            joints_3d_visible = np.zeros((num_joints, 3), dtype=np.float32)
            joints_3d_visible[:, :2] = np.minimum(1, keypoints[:, 2:3])

            sample = {
                'image_file': image_file,
                'bbox': bbox,
                'rotation': 0,
                'joints_3d': joints_3d,
                'joints_3d_visible': joints_3d_visible,
                'bbox_score': 1,
                'bbox_id': idx,
            }
            db.append(sample)

        return db

    def evaluate(self, results, res_folder=None, metric='PCK', **kwargs):
        """Evaluate keypoint detection results. The pose prediction results will
        be saved in `${res_folder}/result_keypoints.json`.

        Note:
        batch_size: N
        num_keypoints: K
        heatmap height: H
        heatmap width: W

        Args:
        results (list(preds, boxes, image_path, output_heatmap))
            :preds (np.ndarray[N,K,3]): The first two dimensions are
                coordinates, score is the third dimension of the array.
            :boxes (np.ndarray[N,6]): [center[0], center[1], scale[0]
                , scale[1],area, score]
            :image_paths (list[str]): For example, ['Test/source/0.jpg']
            :output_heatmap (np.ndarray[N, K, H, W]): model outputs.

        res_folder (str, optional): The folder to save the testing
                results. If not specified, a temp folder will be created.
                Default: None.
        metric (str | list[str]): Metric to be performed.
            Options: 'PCK', 'NME'.

        Returns:
            dict: Evaluation results for evaluation metric.
        """

        metrics = metric if isinstance(metric, list) else [metric]
        allowed_metrics = ['PCK', 'NME']
        for metric in metrics:
            if metric not in allowed_metrics:
                raise KeyError(f'metric {metric} is not supported')

        if res_folder is not None:
            tmp_folder = None
            res_file = osp.join(res_folder, 'result_keypoints.json')
        else:
            tmp_folder = tempfile.TemporaryDirectory()
            res_file = osp.join(tmp_folder.name, 'result_keypoints.json')

        kpts = []
        for result in results:
            preds = result['preds']
            boxes = result['boxes']
            image_paths = result['image_paths']
            bbox_ids = result['bbox_ids']

            batch_size = len(image_paths)
            for i in range(batch_size):
                kpts.append({
                    'keypoints': preds[i].tolist(),
                    'center': boxes[i][0:2].tolist(),
                    'scale': boxes[i][2:4].tolist(),
                    'area': float(boxes[i][4]),
                    'score': float(boxes[i][5]),
                    'bbox_id': bbox_ids[i]
                })
        kpts = self._sort_and_unique_bboxes(kpts)

        self._write_keypoint_results(kpts, res_file)
        info_str = self._report_metric(res_file, metrics)
        name_value = OrderedDict(info_str)

        if tmp_folder is not None:
            tmp_folder.cleanup()

        return name_value

    def _report_metric(self, res_file, metrics, pck_thr=0.3):
        """Keypoint evaluation.

        Args:
        res_file (str): Json file stored prediction results.
        metrics (str | list[str]): Metric to be performed.
            Options: 'PCK', 'NME'.
        pck_thr (float): PCK threshold, default: 0.3.

        Returns:
        dict: Evaluation results for evaluation metric.
        """
        info_str = []

        with open(res_file, 'r') as fin:
            preds = json.load(fin)
        assert len(preds) == len(self.db)

        outputs = []
        gts = []
        masks = []

        for pred, item in zip(preds, self.db):
            outputs.append(np.array(pred['keypoints'])[:, :-1])
            gts.append(np.array(item['joints_3d'])[:, :-1])
            masks.append((np.array(item['joints_3d_visible'])[:, 0]) > 0)

        outputs = np.array(outputs)
        gts = np.array(gts)
        masks = np.array(masks)

        normalize_factor = self._get_normalize_factor(gts)

        if 'PCK' in metrics:
            _, pck, _ = keypoint_pck_accuracy(outputs, gts, masks, pck_thr,
                                              normalize_factor)
            info_str.append(('PCK', pck))

        if 'NME' in metrics:
            info_str.append(
                ('NME', keypoint_nme(outputs, gts, masks, normalize_factor)))

        return info_str

    @staticmethod
    def _write_keypoint_results(keypoints, res_file):
        """Write results into a json file."""

        with open(res_file, 'w') as f:
            json.dump(keypoints, f, sort_keys=True, indent=4)

    @staticmethod
    def _sort_and_unique_bboxes(kpts, key='bbox_id'):
        """sort kpts and remove the repeated ones."""
        kpts = sorted(kpts, key=lambda x: x[key])
        num = len(kpts)
        for i in range(num - 1, 0, -1):
            if kpts[i][key] == kpts[i - 1][key]:
                del kpts[i]

        return kpts

    @staticmethod
    def _get_normalize_factor(gts):
        """Get inter-ocular distance as the normalize factor, measured as the
        Euclidean distance between the outer corners of the eyes.

        Args:
            gts (np.ndarray[N, K, 2]): Groundtruth keypoint location.

        Return:
            np.ndarray[N, 2]: normalized factor
        """

        interocular = np.linalg.norm(
            gts[:, 0, :] - gts[:, 1, :], axis=1, keepdims=True)
        return np.tile(interocular, [1, 2])


KeyError: ignored

## Configure mmpose

### Create a config file

In the next step, we create a config file which configures the model, dataset and runtime settings. More information can be found at [Learn about Configs](https://mmpose.readthedocs.io/en/latest/tutorials/0_config.html). A common practice to create a config file is deriving from a existing one. In this tutorial, we load a config file that trains a HRNet on COCO dataset, and modify it to adapt to the COCOTiny dataset.

In [12]:
!rm cpm_config.py
!gdown "1FW41QIxH0CMxvSXTxcbZgQoeDRV1LUw_&confirm=t" # https://drive.google.com/file/d/1FW41QIxH0CMxvSXTxcbZgQoeDRV1LUw_/view?usp=share_link

Downloading...
From: https://drive.google.com/uc?id=1FW41QIxH0CMxvSXTxcbZgQoeDRV1LUw_&confirm=t
To: /content/mmpose/cpm_config.py
100% 4.44k/4.44k [00:00<00:00, 7.05MB/s]


### Config for coco tiny

In [17]:
from mmcv import Config


cfg = Config.fromfile('/content/mmpose/cpm_config.py')


# set basic configs
cfg.data_root = 'data/coco_tiny'
cfg.work_dir = 'work_dirs/cpm_coco_tiny_256x192'
cfg.gpu_ids = range(1)
cfg.seed = 0

# set log interval
cfg.log_config.interval = 1

# set evaluation configs
cfg.evaluation.interval = 1
cfg.evaluation.metric = 'PCK'
cfg.evaluation.save_best = 'PCK'

cfg.total_epochs = 2

# set batch size
cfg.data.samples_per_gpu = 32
cfg.data.val_dataloader = dict(samples_per_gpu=32)
cfg.data.test_dataloader = dict(samples_per_gpu=32)

# set dataset configs
cfg.data.train.type = 'TopDownCOCOTinyDataset5'
cfg.data.train.ann_file = f'{cfg.data_root}/train.json'
cfg.data.train.img_prefix = f'{cfg.data_root}/images/'

cfg.data.val.type = 'TopDownCOCOTinyDataset5'
cfg.data.val.ann_file = f'{cfg.data_root}/val.json'
cfg.data.val.img_prefix = f'{cfg.data_root}/images/'

cfg.data.test.type = 'TopDownCOCOTinyDataset5'
cfg.data.test.ann_file = f'{cfg.data_root}/val.json'
cfg.data.test.img_prefix = f'{cfg.data_root}/images/'

#print(cfg.pretty_text)


## Train and Evaluation


In [18]:
from mmpose.datasets import build_dataset
from mmpose.models import build_posenet
from mmpose.apis import train_model
import mmcv

# build dataset
datasets = [build_dataset(cfg.data.train)]
# # build model
model = build_posenet(cfg.model)
# create work_dir
mmcv.mkdir_or_exist(cfg.work_dir)

In [19]:
# Helper function for counting number of trainable parameters.
def count_params(model):
    """
    Counts the number of trainable parameters in PyTorch.
    Args:
        model: PyTorch model.
    Returns:
        num_params: int, number of trainable parameters.
    """
    num_params = sum([item.numel() for item in model.parameters() if item.requires_grad])
    return num_params

In [20]:
print('Number of trainable parameters {}'.format(count_params(model)))

Number of trainable parameters 31304166


In [21]:
# # train model
train_model(model, datasets, cfg, distributed=False, validate=True, meta=dict())


2022-12-11 21:46:46,094 - mmpose - INFO - Start running, host: root@ab41507c2ae7, work_dir: /content/mmpose/work_dirs/cpm_coco_tiny_256x192
2022-12-11 21:46:46,096 - mmpose - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(ABOVE_NORMAL) Fp16OptimizerHook                  
(NORMAL      ) CheckpointHook                     
(NORMAL      ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) EvalHook                           
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) EvalHook                           
(LOW         ) IterTimerHook                      
 -------------------- 
after_train_iter:
(ABOVE_NORMAL) Fp16Op

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 25/25, 25.1 task/s, elapsed: 1s, ETA:     0s

2022-12-11 21:46:55,391 - mmpose - INFO - Now best checkpoint is saved as best_PCK_epoch_1.pth.
2022-12-11 21:46:55,395 - mmpose - INFO - Best PCK is 0.0084 at 1 epoch.
2022-12-11 21:46:55,397 - mmpose - INFO - Epoch(val) [1][1]	PCK: 0.0084
2022-12-11 21:46:59,446 - mmpose - INFO - Epoch [2][1/3]	lr: 3.497e-06, eta: 0:00:04, time: 4.038, data_time: 2.798, memory: 4737, heatmap_loss: 0.0574, acc_pose: 0.0049, loss: 0.0574
2022-12-11 21:47:00,623 - mmpose - INFO - Epoch [2][2/3]	lr: 4.496e-06, eta: 0:00:02, time: 1.177, data_time: 0.005, memory: 4737, heatmap_loss: 0.0461, acc_pose: 0.0105, loss: 0.0461
2022-12-11 21:47:00,959 - mmpose - INFO - Epoch [2][3/3]	lr: 5.495e-06, eta: 0:00:00, time: 0.336, data_time: 0.004, memory: 4737, heatmap_loss: 0.0466, acc_pose: 0.0000, loss: 0.0466
2022-12-11 21:47:01,095 - mmpose - INFO - Saving checkpoint at 2 epochs


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 25/25, 20.6 task/s, elapsed: 1s, ETA:     0s

2022-12-11 21:47:04,687 - mmpose - INFO - The previous best checkpoint /content/mmpose/work_dirs/cpm_coco_tiny_256x192/best_PCK_epoch_1.pth was removed
2022-12-11 21:47:06,912 - mmpose - INFO - Now best checkpoint is saved as best_PCK_epoch_2.pth.
2022-12-11 21:47:06,915 - mmpose - INFO - Best PCK is 0.0310 at 2 epoch.
2022-12-11 21:47:06,919 - mmpose - INFO - Epoch(val) [2][1]	PCK: 0.0310
